In [ ]:
using CSV
using DataFrames
using CairoMakie
using Colors
using ColorSchemes

const global_scheme = ColorSchemes.seaborn_pastel

In [ ]:
function plot_latencies()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Latency [μs]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "latency"] * 1.0e6, label=first(dft[!,"type"]), linestyle=:dot)
            push!(base_data, d)
        else
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "latency"] * 1.0e6, label="$(first(dft[!,"type"])) Byte", linestyle=:dash)
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "latency"] * 1.0e6, label="MPI", linestyle=:dot))
    
    df_aurora_streaming = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Streaming"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_streaming[!, "MSize"], df_aurora_streaming[!, "latency"] * 1.0e6, label="Aurora/Streaming", linestyle=:solid))

    df_aurora_framing = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Framing"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_framing[!, "MSize"], df_aurora_framing[!, "latency"] * 1.0e6, label="Aurora/Framing", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "latency"] * 1.0e6, label="SerialLite (2x 40G)", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^28))
    ylims!(ax_udp, (10^-1, 10^5))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["UDP Max. Payload", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

fig = plot_latencies()
save("latencies.pdf", fig)
fig

In [ ]:
function plot_throughput()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=identity,
        xlabel="Message Size [Byte]", ylabel="Throughput [Gbit/s]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []

    df[!, "throughput"] = df[!, "MSize"] ./ df[!, "latency"] * 8 / 1000000000
    
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "throughput"], label=first(dft[!,"type"]), linestyle=:dot)
            push!(base_data, d)
        else
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "throughput"], label=first(dft[!,"type"]), linestyle=:dash)
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "throughput"], label="MPI", linestyle=:dot))
    
    df_aurora_streaming = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Streaming"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_streaming[!, "MSize"], df_aurora_streaming[!, "throughput"], label="Aurora/Streaming", linestyle=:solid))

    df_aurora_framing = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Framing"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_framing[!, "MSize"], df_aurora_framing[!, "throughput"], label="Aurora/Framing", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "throughput"], label="SerialLite (2 x 40G)", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^28))
    ylims!(ax_udp, (0, 10^2))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["UDP", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

fig = plot_throughput()
save("throughput.pdf", fig)
fig 

In [ ]:
function plot_over_message_size(message_size)
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    df = sort(df[(df.MSize .== message_size),:], rev=true)
    delete!(df, 1)
    barplot(
        fig[1,1],
        df[!, "latency"] * 1.0e6,
        bar_labels = :y,
        axis=(
            xlabel="Communication Type", ylabel="Latency [μs]",
            yminorticksvisible=true, yminorgridvisible=true,
            yminorticks=IntervalsBetween(5),
            xticks = (1:7, ["ACCL PL Stream", "MPI", "", "", "SerialLite", "Aurora\nFraming", "Aurora\nStreaming"]),
        ),
    )
    fig
end

fig = plot_over_message_size(64)
save("latency_64.pdf", fig)
fig

In [ ]:
function plot_throughput_over_frame_sizes()
    df = CSV.read("latency_over_frame_sizes.csv", DataFrame)
    df_udp = CSV.read("data.csv", DataFrame)
    filter!(x -> x.protocol == "UDP" && x.type != "MPI+PCIe", df_udp)
    df_udp[!, "throughput"] = df_udp[!, "MSize"] ./ df_udp[!, "latency"] * 8 / 1000000000
    sort!(df_udp, :type, by=x->parse(Int,x))
    fig = Figure()
    ax = Axis(fig[1, 1], xscale=log2, #yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Throughput [Gbit/s]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    data = []
    frame_sizes = [2,4,7,10]
    message_sizes = df[:, 1]

    udp_data = [] 
    types = sort(unique(df_udp[!,"type"]), by=x->parse(Int,x))
    typecolor = Dict(t => global_scheme[i] for (i,t) in enumerate(types))
    for (i, dft) in enumerate(groupby(df_udp,"type"))
        d = lines!(ax, dft[!, "MSize"], dft[!, "throughput"], label="$(first(dft[!,"type"])) Byte", linestyle=:dash, color=typecolor[first(dft[!,"type"])])
        push!(udp_data, d)
    end

    for (c,i) in enumerate(frame_sizes)
        frame_size = 2^i
        df_frame = dropmissing(df[!, ["message_size", string(frame_size)]])
        valid_message_sizes = 2.0 .^(i:i+nrow(df_frame)-1) .* 64
        valid_df_frame = valid_message_sizes ./ df_frame[!,string(frame_size)] .* 8 ./ 1000000000
        #display(valid_df_frame)
        push!(data, lines!(ax, valid_message_sizes, valid_df_frame, label="$(frame_size * 64) Byte", color=global_scheme[c]))
    end
    
    xlims!(ax, (2^6, 2^25))
    ylims!(ax, (0, 10^2))
    Legend(fig[1, 2], [data, reverse(udp_data)], [[l.label for l in data], reverse([l.label for l in udp_data])], ["Frame Size", "UDP Payload"], nbanks=1, orientation=:vertical)
    fig
end

fig = plot_throughput_over_frame_sizes()
save("throughput_over_frame_sizes.pdf", fig)
fig

In [ ]:
function plot_latencies_over_frame_sizes()
    df = CSV.read("latency_over_frame_sizes.csv", DataFrame)
    fig = Figure()
    ax = Axis(fig[1, 1], xscale=log2, yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Latency [μs]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    data = []
    frame_sizes = [2^n for n in 0:22]
    message_sizes = df[:, 1]

    for (i, frame_size) in enumerate(frame_sizes)
        df_frame = df[:, i+1]
        valid_indices = findall(!ismissing, df_frame)
        valid_message_sizes = message_sizes[valid_indices]
        valid_df_frame = Float64.(df_frame[valid_indices]) .* 1e+6
        #display(valid_df_frame)
        push!(data, lines!(ax, valid_message_sizes, valid_df_frame, label=string(frame_size)))
    end
    
    xlims!(ax, (2^6, 2^20))
    ylims!(ax, (10^0, 10^2))
    Legend(fig[1, 2], data, [l.label for l in data], ["Frame Size"], nbanks=1, orientation=:vertical)
    fig
end

fig = plot_latencies_over_frame_sizes()
save("latencies_over_frame_sizes.pdf", fig)
fig

In [ ]:
max_res() = Dict(
    "LUT" => 1179377,
    "REG" => 591488,
    "URAM" => 2436368,
    "BRAM" => 1819,
    "DSP" => 9020
)

group_names() = ["Platform", "Network", "ACCL", "User", "Aurora"]

function prepare_resources_dataframe(data_csv)
    df = CSV.read(data_csv, DataFrame)
    df[!, "color"] = [
        if any(contains.(d, ["cmac", "network"]))
            2
        elseif any(contains.(d, ["recv", "send"]))
            4
        elseif any(contains.(d, ["Platform"]))
            1
        elseif any(contains.(d, ["aurora_hls"]))
	    5
	else
            3
        end for d in df[!, "Name"]
    ]
    df_new = DataFrame()
    for i in ["LUT", "BRAM", "REG", "DSP"] # skip URAM
        df_tmp = df[!, ["color", "Name", "bitstream"]]
        df_tmp[!, "type"] .= i
        df_tmp[!, "value_abs"] = df[!, i]
        df_tmp[!, "value"] = df[!, i] ./ max_res()[i]
        append!(df_new, df_tmp)
    end
    sort!(df_new, [:color, :bitstream])
    df = df_new[.!contains.(df_new[!, "Name"], "User Budget"), :]
    df = df[.!contains.(df[!, "Name"], "Platform"), :]
    df = combine(groupby(df, [:type, :color, :bitstream]), names(df, Not(:type, :color, :bitstream, :Name)) .=> sum, renamecols=false)
    df[!, "Name"] = [group_names()[c] for c in df.color]
    df
    # df[contains.(df[!, "Name"], "Platform"), :]
end

function plot_resources()
    df = prepare_resources_dataframe("resources.csv")
    unique_names = unique(df[!, "Name"])
    fig = Figure(resolution=(480, 400), fontsize=10)
    categories = ["Aurora Streaming (32 Bytes Width)", "Aurora Framing (32 Bytes Width)", "Aurora Streaming (64 Bytes Width)", "Aurora Framing (64 Bytes Width)"]
    types = ["Aurora"]
    ax = Axis(fig[1, 1], xlabel="Utilization [%]", yticks=(1:length(["LUT", "BRAM", "REG", "DSP"]), ["LUT", "BRAM", "REG", "DSP"]),
        limits=((0,3.5), nothing),
        palette=(patchcolor=global_scheme,))
    hidespines!(ax, :t, :r, :l)
    hideydecorations!(ax, ticklabels=false)

    aurora_streaming_32 = df[in.(df.Name, (["Aurora"],)).&&endswith.(df.bitstream, "_0_32"), :]
    aurora_streaming_32[!, "category"] .= "Aurora Streaming (32 Bytes Width)"

    aurora_framing_32 = df[in.(df.Name, (["Aurora"],)).&&endswith.(df.bitstream, "_1_32"), :]
    aurora_framing_32[!, "category"] .= "Aurora Framing (32 Bytes Width)"

    aurora_streaming_64 = df[in.(df.Name, (["Aurora"],)).&&endswith.(df.bitstream, "_0_64"), :]
    aurora_streaming_64[!, "category"] .= "Aurora Streaming (64 Bytes Width)"

    aurora_framing_64 = df[in.(df.Name, (["Aurora"],)).&&endswith.(df.bitstream, "_1_64"), :]
    aurora_framing_64[!, "category"] .= "Aurora Framing (64 Bytes Width)"

    append!(aurora_streaming_32, aurora_framing_32)
    append!(aurora_streaming_32, aurora_streaming_64)
    append!(aurora_streaming_32, aurora_framing_64)
    df = aurora_streaming_32
     
    print(df)

    barplot!(ax, [findfirst(x -> x == t, ["LUT", "BRAM", "REG", "DSP"]) for t in df.type],
        df.value .* 100,
        direction=:x,
        color=[global_scheme[findfirst(x -> x == n, categories)] for n in df.category],
        stack=[findfirst(x -> x == n, types) for n in df.Name],
        dodge=[findfirst(x -> x == n, categories) for n in df.category],
        flip_labels_at=10,
        label_size=10,
        bar_labels=[sum(df[(df.category.==r.category).&(df.type.==r.type), "value_abs"]) for r in eachrow(df)]
    )
    # Legend
    elements = [PolyElement(polycolor=global_scheme[i]) for i in 1:length(categories[1:end])]
    title = "Legend"

    Legend(fig[1, 2], reverse(elements), reverse(categories), title)
    fig
end

fig = plot_resources()
save("resources.pdf", fig)
fig